In [ ]:
# Chat with in memory history
import os
from typing import Dict
from langchain_ollama import OllamaLLM
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


# -------------------------------
# Global configuration
# -------------------------------
OLLAMA_URL = os.getenv("OLLAMA_URL", "http://127.0.0.1:11434")


def setup_environment() -> None:
    """Ensure local Ollama traffic bypasses proxies."""
    os.environ.setdefault("no_proxy", "127.0.0.1,localhost")
    os.environ.setdefault("HTTPX_NO_PROXY", "127.0.0.1,localhost")


def create_chat_chain(model_name: str):
    """
    Create a conversational chain with in-memory history.

    Args:
        model_name: Name of the Ollama model to use.

    Returns:
        chain_with_history: RunnableWithMessageHistory instance
        history_store: Dict holding chat histories by session_id
    """
    # Initialize Ollama LLM
    llm = OllamaLLM(model=model_name, base_url=OLLAMA_URL, timeout=30)

    # Define prompt template
    prompt = ChatPromptTemplate.from_messages([
        ("system",
         "You are a helpful AI assistant. "
         "Respond in Persian if the user speaks Persian, otherwise in English."),
        MessagesPlaceholder(variable_name="messages"),
        ("human", "{content}")
    ])

    # Base chain (prompt → model)
    chain = prompt | llm

    # Session-based history storage
    history_store: Dict[str, InMemoryChatMessageHistory] = {}

    def get_session_history(session_id: str):
        """Retrieve or create chat history for the session."""
        if session_id not in history_store:
            history_store[session_id] = InMemoryChatMessageHistory()
        return history_store[session_id]

    # Add history management
    chain_with_history = RunnableWithMessageHistory(
        chain,
        get_session_history,
        input_messages_key="content",
        history_messages_key="messages"
    )

    return chain_with_history, history_store


def main():
    """Run an interactive chat with persistent in-memory history."""
    setup_environment()

    model_name = "llama3.2:3b"
    chain_with_history, history_store = create_chat_chain(model_name)

    print("Start chatting (type 'exit' to quit):")
    session_id = "default_session"

    while True:
        user_input = input(">> ").strip()
        if user_input.lower() == "exit":
            print("Goodbye!")
            break

        try:
            # Run the chain
            response = chain_with_history.invoke(
                {"content": user_input, "messages": []},
                config={"configurable": {"session_id": session_id}}
            )

            print(response.strip())

        except Exception as e:
            print(f"⚠️ Error: {e}")

        # Debug: show current history (optional, can be removed)
        # print("History:", history_store[session_id].messages)


if __name__ == "__main__":
    main()


Start chatting (type 'exit' to quit):
English

The answer is 2.
English

The result of adding 3 to 2 is 5.
Goodbye!


In [ ]:
# Chat with saving history in Json
import os
from typing import Dict
from langchain_ollama import OllamaLLM
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import FileChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Global Ollama configuration
OLLAMA_URL = os.getenv("OLLAMA_URL", "http://127.0.0.1:11434")
HISTORY_DIR = "chat_history"

def setup_environment() -> None:
    """Ensure local Ollama traffic bypasses proxies and create history directory."""
    os.environ.setdefault("no_proxy", "127.0.0.1,localhost")
    os.environ.setdefault("HTTPX_NO_PROXY", "127.0.0.1,localhost")
    os.makedirs(HISTORY_DIR, exist_ok=True)  # Create history directory if it doesn't exist

def create_chat_chain(model_name: str) -> tuple[RunnableWithMessageHistory, Dict]:
    """
    Create a conversational chain with persistent file-based history.

    Args:
        model_name: Name of the Ollama model to use.

    Returns:
        chain_with_history: RunnableWithMessageHistory instance
        history_store: Dict holding chat histories by session_id
    """
    try:
        # Initialize Ollama LLM
        llm = OllamaLLM(model=model_name, base_url=OLLAMA_URL, timeout=30)

        # Define prompt template
        prompt = ChatPromptTemplate.from_messages([
            ("system",
             "You are a helpful AI assistant. "
             "Respond in Persian if the user speaks Persian, otherwise in English."),
            MessagesPlaceholder(variable_name="messages"),
            ("human", "{content}")
        ])

        # Base chain (prompt → model)
        chain = prompt | llm

        # Session-based history storage
        history_store: Dict[str, FileChatMessageHistory] = {}

        def get_session_history(session_id: str) -> FileChatMessageHistory:
            """Retrieve or create chat history for the session."""
            history_file = os.path.join(HISTORY_DIR, f"chat_history_{session_id}.json")
            if session_id not in history_store:
                history_store[session_id] = FileChatMessageHistory(history_file)
            return history_store[session_id]

        # Add history management
        chain_with_history = RunnableWithMessageHistory(
            chain,
            get_session_history,
            input_messages_key="content",
            history_messages_key="messages"
        )

        return chain_with_history, history_store

    except Exception as e:
        raise

def main():
    """Run an interactive chat with persistent file-based history."""
    setup_environment()

    model_name = "llama3.2:3b"

    # Create chat chain and history store
    try:
        chain_with_history, history_store = create_chat_chain(model_name)
    except Exception as e:
        return

    print("Start chatting (type 'exit' to quit):")
    session_id = "default_session"

    while True:
        user_input = input(">> ").strip()
        if user_input.lower() == "exit":
            print("Goodbye!")
            break

        try:
            # Run the chain
            response = chain_with_history.invoke(
                {"content": user_input, "messages": []},
                config={"configurable": {"session_id": session_id}}
            )

            print(response.strip())

            # Debug: show current history (optional, can be removed)
            # print("History:", history_store[session_id].messages)

        except Exception as e:
            print(f"⚠️ Error: {e}")

if __name__ == "__main__":
    main()

2025-09-04 17:04:46,256 - INFO - Created or verified history directory: chat_history
2025-09-04 17:04:46,257 - INFO - 🚀 Initializing model: llama3.2:3b


2025-09-04 17:04:46,808 - INFO - Initialized LLM: llama3.2:3b


Start chatting (type 'exit' to quit):


2025-09-04 17:04:54,026 - INFO - Initialized history file for session default_session: chat_history\chat_history_default_session.json
2025-09-04 17:04:58,598 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
2025-09-04 17:04:59,596 - INFO - ✅ Generated response


پیش بینی می شود که 1+1 = 2 است.


2025-09-04 17:05:30,937 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
2025-09-04 17:05:32,246 - INFO - ✅ Generated response


از طریق این toán، به این نتیجه می رسیم که 2 + 3 = 5 است.


2025-09-04 17:05:49,195 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
2025-09-04 17:05:49,680 - INFO - ✅ Generated response


آشنا میشم. goodbye.
Goodbye!


In [ ]:
# Chat with saving history in SQLite
import os
from typing import Dict
from sqlalchemy import create_engine
from langchain_ollama import OllamaLLM
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


# -------------------------------
# Global configuration
# -------------------------------
OLLAMA_URL = os.getenv("OLLAMA_URL", "http://127.0.0.1:11434")
DB_PATH = "chat_history.db"  # SQLite database file
engine = create_engine(f"sqlite:///{DB_PATH}")


def setup_environment() -> None:
    """Ensure local Ollama traffic bypasses proxies."""
    os.environ.setdefault("no_proxy", "127.0.0.1,localhost")
    os.environ.setdefault("HTTPX_NO_PROXY", "127.0.0.1,localhost")


def create_chat_chain(model_name: str):
    """
    Create a conversational chain with SQLite-backed history.

    Args:
        model_name: Name of the Ollama model to use.

    Returns:
        chain_with_history: RunnableWithMessageHistory instance
        history_store: Dict holding chat histories by session_id
    """
    # Initialize Ollama LLM
    llm = OllamaLLM(model=model_name, base_url=OLLAMA_URL, timeout=30)

    # Define prompt template
    prompt = ChatPromptTemplate.from_messages([
        ("system",
         "You are a helpful AI assistant. "
         "Respond in Persian if the user speaks Persian, otherwise in English."),
        MessagesPlaceholder(variable_name="messages"),
        ("human", "{content}")
    ])

    # Base chain (prompt → model)
    chain = prompt | llm

    # Session-based history storage
    history_store: Dict[str, SQLChatMessageHistory] = {}

    def get_session_history(session_id: str):
        """Retrieve or create chat history for the session."""
        if session_id not in history_store:
            history_store[session_id] = SQLChatMessageHistory(
                session_id=session_id,
                connection=engine
            )
        return history_store[session_id]

    # Add history management
    chain_with_history = RunnableWithMessageHistory(
        chain,
        get_session_history,
        input_messages_key="content",
        history_messages_key="messages"
    )

    return chain_with_history, history_store


def main():
    """Run an interactive chat with persistent SQLite history."""
    setup_environment()

    model_name = "llama3.2:3b"
    chain_with_history, history_store = create_chat_chain(model_name)

    print("Start chatting (type 'exit' to quit):")
    session_id = "default_session"

    while True:
        user_input = input(">> ").strip()
        if user_input.lower() == "exit":
            print("Goodbye!")
            break

        try:
            # Run the chain
            response = chain_with_history.invoke(
                {"content": user_input, "messages": []},
                config={"configurable": {"session_id": session_id}}
            )

            print(response.strip())

        except Exception as e:
            print(f"⚠️ Error: {e}")


if __name__ == "__main__":
    main()


Start chatting (type 'exit' to quit):


2025-09-04 17:11:23,867 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


English.

The answer is 2.


2025-09-04 17:11:46,724 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


English.

The result was 2. Adding 3 gives us: 2 + 3 = 5.


2025-09-04 17:12:05,429 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


خداحافظ! (Goodbye!)

Note: I responded in Persian to acknowledge your initial message, but switched back to English as per our initial agreement since the majority of our conversation was in English. If you'd like to switch back to Persian or have any further questions, feel free to ask!
Goodbye!
